In [ ]:
from astrocast import analysis

In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import napari
import dask.array as da
from skimage.filters import threshold_triangle


In [ ]:
vid = analysis.Video("/media/janrei1/data/astrocyte_examples/31570865_AquA/GlusnfrSuppRaw.h5", h5_loc="dff/ch0", lazy=True, name="dff")
arr = vid.get_data()#.compute()

arr = arr.rechunk(("auto", "auto", "auto"))
arr

In [ ]:
radius = 3
sigma = 3

def func(arr):
    return scipy.ndimage.gaussian_filter(arr, sigma=sigma, radius=radius, mode='nearest')

depth = radius*2 + 1
print(f"depth: {depth}")
depth = {0:depth, 1:depth, 2:depth}
print(f"depth: {depth}")

display(arr)

overlap = da.overlap.overlap(arr, depth=depth, boundary="nearest")
display(overlap)

mapped = overlap.map_blocks(func, dtype=arr.dtype)
display(mapped)

arr2 = da.overlap.trim_internal(mapped, depth, boundary='nearest')
display(arr2)

# arr2 = arr2.compute()

In [ ]:
plt.imshow(arr2[42, :, :])

In [ ]:
def showcase(arr, arr2, z, figsize=(15, 15)):

    if isinstance(arr, da.Array):
        arr = arr.compute()

    if isinstance(arr2, da.Array):
        arr = arr.compute()
        
    fig, axx = plt.subplots(2, 2, figsize=(5, 3))

    img1 = arr[z, :, :]
    img1t = img1 > threshold_triangle(img1)
    
    axx[0, 0].imshow(img1)
    axx[0, 1].imshow(img1t)


    img2 = arr2[z, :, :]
    img2t = img2 > threshold_triangle(img2)
    
    axx[1, 0].imshow(img2)
    axx[1, 1].imshow(img2t)

    plt.tight_layout()

showcase(arr, arr2, z=42)
showcase(arr, arr2, z=175)

In [ ]:
showcase(arr[:, :, -100:], arr2[:, :, -100:], z=41)

In [ ]:
chunk_size = (1, 50, 50)

def dynamic_threshold(arr, min_ratio=1):

    # if len(arr) % 2 == 0:
    #     raise ValueError(f"Sequence length is not odd: {len(arr)}")

    # calculate threshold
    threshold = threshold_triangle(arr)

    # threshold image
    # center_index = (len(arr) - 1) // 2
    # imc = arr[center_index, :, :]
    # active_pixels = imc > threshold
    active_pixels = arr > threshold
    
    # calculate ratio foreground/background
    active_ind = np.where(active_pixels == 1) # TODO more efficient solution?
    inactive_ind = np.where(active_pixels == 0)

    # fg = np.mean(imc[active_ind])
    # bg = abs(np.mean(imc[inactive_ind]))
    # ratio = fg/bg
    
    fg = np.mean(arr[active_ind])
    bg = abs(np.mean(arr[inactive_ind]))
    ratio = fg/bg
    
    # adjust axis
    # active_pixels = active_pixels.reshape((1,) + active_pixels.shape)

    if ratio > min_ratio:
        return active_pixels
    else:
        return np.zeros(active_pixels.shape, np.bool_)

threshold_z_depth = 1
data = arr2.copy()
display(data)

# depth = (threshold_z_depth, 10, 10)
depth = {0:threshold_z_depth, 1:10, 2:10}
data = data.rechunk(chunk_size)
display(data)
overlap = da.overlap.overlap(data, depth=depth, boundary="nearest")
display(overlap)

mapped = overlap.map_blocks(dynamic_threshold, dtype=data.dtype)
display(mapped)

active_pixels = da.overlap.trim_internal(mapped, depth, boundary='nearest')
display(active_pixels)

active_pixels = active_pixels.compute()

# active_pixels = overlap.map_blocks(dynamic_threshold,
#                                    chunks=chunk_size, dtype=data.dtype)

fig, ax = plt.subplots(1, 1)
ax.imshow(active_pixels[42, :, :])

In [ ]:
n = 4
for A in (arr, arr2):
    for i in (8, 140, 174):    
        im = A[i:i+n, :, :]#.compute()
    
        if isinstance(im, da.Array):
            im = im.compute()
        
        thresh = threshold_triangle(im)
        imt = im > thresh
    
        bck = np.mean(im[np.where(imt == 0)])
        fg = np.mean(im[np.where(imt == 1)])
        title = f"bck: {bck:.2f} fg: {fg:.2f} ratio: {-1*fg/bck:.2f}"
        
        fig, axx = plt.subplots(1, len(im), figsize=(15, 15))
        for z in range(len(imt)):
            axx[z].imshow(imt[z, :, :])
    
        axx[0].set_title(title)

In [ ]:
from tqdm import tqdm

def threshold(arr, n=1, min_ratio=1):

    if isinstance(arr, da.Array):
        arr = arr.compute()

    res = np.zeros(arr.shape, dtype=arr.dtype)
    print(f"0: {np.mean(res)}")
    
    print(arr.shape)
    arr = np.pad(arr, pad_width=((n, n), (0, 0), (0, 0)), mode="edge")
    print(arr.shape)
    
    for z in tqdm(range(n, len(arr)-1)):

        ratio = -1 
        
        im = arr[z-n:z+n, :, :]
        threshold = threshold_triangle(im)
        
        imc = im[n, :, :]
        active_pixels = imc > threshold 

        active_ind = np.where(active_pixels == 1)
        inactive_ind = np.where(active_pixels == 0)
        
        fg = np.mean(imc[active_ind])
        bg = abs(np.mean(imc[inactive_ind]))
        ratio = fg/bg
        # print(ratio)
        if ratio > min_ratio:
            print(z, ratio)
            res[z-n, :, :] = active_pixels

    return res

act = threshold(arr2, n=1)
act.shape

In [ ]:
viewer = napari.Viewer()
viewer.add_image(act, colormap="gray")
viewer.show()

In [ ]:
def dynamic_threshold(arr, min_ratio=1):
    
    if len(arr) % 2 == 0:
        raise ValueError("Sequence length is not odd")
    
    # calculate threshold
    threshold = threshold_triangle(arr)

    # threshold image
    center_index = (len(arr) - 1) // 2
    imc = arr[center_index, :, :]
    active_pixels = imc > threshold 

    # calculate ratio foreground/background
    active_ind = np.where(active_pixels == 1)
    inactive_ind = np.where(active_pixels == 0)
    
    fg = np.mean(imc[active_ind])
    bg = abs(np.mean(imc[inactive_ind]))
    ratio = fg/bg
    print(fg, bg, ratio)

    # adjust axis
    active_pixels = active_pixels.reshape((1,) + active_pixels.shape)
    
    if ratio > min_ratio:
        return active_pixels
    else:
        return np.zeros(active_pixels.shape, np.bool_)

i = 42
im = dynamic_threshold(arr2[i:i+3, :, :].compute(), min_ratio=0.3)

fig, ax = plt.subplots(1, 2)
ax[0].imshow(arr2[i, :, :])
ax[1].imshow(im[0, :, :])

In [ ]:
test = arr.copy()
test = test.rechunk((1, -1, -1))
display(test)

depth = (1, 0, 0)

g = da.overlap.overlap(test, depth=depth, boundary="nearest")
display(g)
g2 = g.map_blocks(dynamic_threshold, chunks=(1, 108, 199), dtype=arr.dtype)
display(g2)

# result = da.overlap.trim_internal(g2, depth)
res = g2.compute()
display(res.shape)

plt.imshow(res[42, :, :])

In [ ]:
import scipy.ndimage as ndimage
from dask_image import ndmorph, ndfilters
import tifffile as tiff

In [ ]:
binary_struct_connectivity= 0
binary_struct_iterations= 2

# struct = ndimage.generate_binary_structure(3, binary_struct_connectivity)
# struct_expanded = ndimage.iterate_structure(struct, binary_struct_iterations).astype(int)

struct = np.ones((3, 3, 3), dtype=np.bool_)
struct[0, :, :] = [[0, 0, 0], [0, 1, 0], [0, 0, 0]]
struct[1, :, :] = [[0, 1, 0], [1, 1, 1], [0, 1, 0]]
struct[2, :, :] = [[0, 0, 0], [0, 1, 0], [0, 0, 0]]
# struct = ndimage.iterate_structure(struct, binary_struct_iterations).astype(np.bool_)

# struct_expanded = np.ones((3, 3, 3), dtype=np.bool_)
# # struct_expanded[0, :, :] = [[0, 0, 0], [0, 1, 0], [0, 0, 0]]
# struct_expanded[1, :, :] = [[0, 0, 0], [0, 1, 0], [0, 0, 0]]
# struct_expanded[2, :, :] = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]

struct_expanded = np.zeros((3, 2, 2), dtype=np.bool_)
struct_expanded[0, :, :] = [[0, 1], [0, 0]]
struct_expanded[1, :, :] = [[1, 0], [1, 0]]
struct_expanded[2, :, :] = [[0, 0], [0, 0]]


N = len(struct)
fig, axx = plt.subplots(1, N)
for n in range(N):
    axx[n].imshow(struct[n, :, :], vmin=0, vmax=1)

N = len(struct_expanded)
fig, axx = plt.subplots(1, N, sharex=True, sharey=True)
for n in range(N):
    axx[n].imshow(struct_expanded[n, :, :], vmin=0, vmax=1)

tiff.imwrite("/media/janrei1/data/astrocyte_examples/31570865_AquA/GlusnfrSuppRaw.roi/struct_.tiff", data=struct)
tiff.imwrite("/media/janrei1/data/astrocyte_examples/31570865_AquA/GlusnfrSuppRaw.roi/struct_expanded_.tiff", data=struct_expanded.astype(bool))

In [ ]:
act = da.from_array(res)

f = 175
figsize=(12, 2)

## Indiviudal

for s, lbl in [(struct, "S1"), (struct_expanded, "S2")]:
    
    act_open = ndmorph.binary_opening(act, structure=struct_expanded)
    act_closed = ndmorph.binary_closing(act, structure=struct_expanded)
    
    act_open = act_open.compute()
    act_closed = act_closed.compute()
    
    fig, axx = plt.subplots(1, 3, figsize=figsize)
    axx[0].imshow(res[f, :, :])
    axx[1].imshow(act_open[f, :, :])
    axx[2].imshow(act_closed[f, :, :])
    
    axx[0].set_title(f"original")
    axx[1].set_title(f"{lbl} open")
    axx[2].set_title(f"{lbl} close")

# Combined
for so, sc, lbl in [
    # (struct, struct, "S1"),
    # (struct_expanded, struct_expanded, "S2"),
    # (struct, struct_expanded, "S1, S2"),
    (struct_expanded, struct, "S2, S1")
]:
    
    open_close = ndmorph.binary_closing(ndmorph.binary_opening(act, structure=so), structure=sc)
    close_open = ndmorph.binary_opening(ndmorph.binary_closing(act, structure=sc), structure=so)
    
    open_close = open_close.compute()
    close_open = close_open.compute()
    
    for f in [f]:
        
        fig, axx = plt.subplots(1, 3, figsize=figsize)
        axx[0].imshow(res[f, :, :])
        axx[1].imshow(open_close[f, :, :])
        axx[2].imshow(close_open[f, :, :])
    
        axx[0].set_title(f"{lbl} ({f})")
        axx[1].set_title("open_close")
        axx[2].set_title("close_open")